## Подготовка данных

In [4]:
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [5]:
data = pd.read_csv(r'C:\Users\Данил\Desktop\Churn.csv')

data.info()
data.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


0

In [13]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [14]:
# заполненим минимальным значением
data['Tenure'] = data['Tenure'].fillna(0)

## Исследование задачи

In [15]:
#Удалим столбцы-идентификаторы, не представляющие ценности
data_ohe = data.drop(['RowNumber','CustomerId', 'Surname'], axis=1)
data_ohe.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
#Разделим на признаки и целевой признак
features = data_ohe.drop('Exited', axis=1)
target = data_ohe['Exited']

features_test = data_ohe.drop('Exited', axis=1)
target_test = data_ohe['Exited']

In [7]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=12345)

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

In [9]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

In [10]:
#Для масштабирования методом scaler зафиксируем численные признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler()

In [11]:
#Масштабируем числ признаки в выборках
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Исследование моделей машинного обучения

In [12]:
model = LogisticRegression()
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)

print('Качество:', accuracy)

Качество: 0.7976


In [13]:
best_accuracy = 0
best_depth = 0
best_est = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_est = est
            
print('Глубина дерева:', best_depth, 'Количество деревьев:', best_est, 'Качество:', best_accuracy)

Глубина дерева: 7 Количество деревьев: 20 Качество: 0.858


In [14]:
best_accuracy = 0
best_depth = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print('Глубина дерева:', best_depth, 'Качество:', best_accuracy)

Глубина дерева: 6 Качество: 0.854


Исследование баланса классов выборки

In [15]:
target_train.value_counts(normalize = 1)

0    0.799733
1    0.200267
Name: Exited, dtype: float64

Как мы выяснили в нашей выборке отрицательны ответов ≈80% , положитительных ≈ 20%. С уверенностью можем сказать что имеется дисбаланс.

Проверим качество моделей до борьбы с дисбалансом

In [16]:
#Заранее напишем функцию для изучия полноты, точности и F1-меры
def rec_prec_f1(target_valid, prediction):
    print("Полнота" , recall_score(target_valid, prediction))
    print("Точность", precision_score(target_valid, prediction))
    print("F1-мера", f1_score(target_valid, prediction))

In [17]:
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 20, max_depth= 7)
model_RFC.fit(features_train, target_train)
RFC_prediction = model_RFC.predict(features_valid)
print(confusion_matrix(target_valid, RFC_prediction))

rec_prec_f1(target_valid, RFC_prediction)

RFC_probabilities_one_valid = model_RFC.predict_proba(features_valid)[:, 1]
auc_roc_RFC = roc_auc_score(target_valid, RFC_probabilities_one_valid)
auc_roc_RFC

[[1920   45]
 [ 310  225]]
Полнота 0.4205607476635514
Точность 0.8333333333333334
F1-мера 0.5590062111801242


0.8613626310908183

Случайный лес примерно равную пропорцию позитивных и негативных предсказаний, выдает заметно больше качественных предсказаний, но также сильно склоняется к ложно позитивным предсказаниям (FP). Точность и качество также низкое.

In [18]:
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth = 6)
model_DTC.fit(features_train, target_train)
DTC_prediction = model_DTC.predict(features_valid)
print(confusion_matrix(target_valid, DTC_prediction))

rec_prec_f1(target_valid, DTC_prediction)

DTC_probabilities_one_valid = model_DTC.predict_proba(features_valid)[:, 1]
auc_roc_DTC = roc_auc_score(target_valid, DTC_probabilities_one_valid)
auc_roc_DTC

[[1917   48]
 [ 317  218]]
Полнота 0.4074766355140187
Точность 0.8195488721804511
F1-мера 0.5443196004993758


0.8463808232860098

Матрица показала, что дерево решений склонно выдавать позитивные предсказания, очень высокое количество ложных позитивных предсказания (FP).

Видим низкое значение F1, следовательно низкое качество модели, проблема в точности.

In [19]:
model_LgR = LogisticRegression()
model_LgR.fit(features_train, target_train)
LgR_prediction = model_LgR.predict(features_valid)
print(confusion_matrix(target_valid, LgR_prediction))

rec_prec_f1(target_valid, LgR_prediction)

LgR_probabilities_one_valid = model_LgR.predict_proba(features_valid)[:, 1]
auc_roc_LgR = roc_auc_score(target_valid, LgR_probabilities_one_valid)
auc_roc_LgR

[[1889   76]
 [ 430  105]]
Полнота 0.19626168224299065
Точность 0.580110497237569
F1-мера 0.2932960893854748


0.758615015100711

## Борьба с дисбалансом

In [20]:
#Создадим функцию для увеличения представленного класса в выборке 
def upsample(features, target, repeat, upsampled_сlass):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    if upsampled_сlass == 0:
        features_upsampled = pd.concat([features_zeros]* repeat + [features_ones] )
        target_upsampled = pd.concat([target_zeros]* repeat + [target_ones] )
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
        
    elif upsampled_сlass == 1:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    else:
        features_upsampled = 0
        target_upsampled = 0  
         
    return features_upsampled, target_upsampled

In [21]:
#Создадим функцию для уменьшения представленной класса в выборке 
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [22]:
#Применим функцию upsample 
#увеличим количество положительных ответов в 4 раза
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 1)
print(target_train_upsampled.value_counts(normalize = 1))
print(target_train_upsampled.shape)

1    0.500416
0    0.499584
Name: Exited, dtype: float64
(12006,)


In [23]:
#Применим функцию downsample
#Уменьшим кол-в пооложительных ответов в 4 раза
#Протестируем функцию (верное значение)
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, 0.2)
print(target_downsampled_train.value_counts(normalize = 0))
print(target_downsampled_train.shape)

1    1502
0    1200
Name: Exited, dtype: int64
(2702,)


выигрышнее выглядит upsample с ним и будем работать

Обучение моделей на сбалансированной выборке

In [24]:
model_RFC_upsampled = RandomForestClassifier(max_depth=7, n_estimators=20, random_state=12345)
model_RFC_upsampled.fit(features_train_upsampled, target_train_upsampled)
RFC_prediction_upsampled = model_RFC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, RFC_prediction_upsampled)

RFC_upsampled_valid = model_RFC_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_RFC = roc_auc_score(target_valid, RFC_upsampled_valid)
auc_roc_RFC

Полнота 0.7401869158878505
Точность 0.5462068965517242
F1-мера 0.6285714285714287


0.859615704739483

In [25]:
model_DTC_upsampled = DecisionTreeClassifier(random_state=12345, max_depth = 6)
model_DTC_upsampled.fit(features_train_upsampled, target_train_upsampled)
DTC_prediction_upsampled = model_DTC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, DTC_prediction_upsampled)

DTC_upsampled_valid = model_DTC_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_DTC = roc_auc_score(target_valid, DTC_upsampled_valid)
auc_roc_DTC

Полнота 0.7700934579439253
Точность 0.4807467911318553
F1-мера 0.5919540229885057


0.8420185013436066

In [26]:
model_LgR_upsampled = LogisticRegression()
model_LgR_upsampled.fit(features_train_upsampled, target_train_upsampled)
LgR_prediction_upsampled = model_LgR_upsampled.predict(features_valid)
rec_prec_f1(target_valid, LgR_prediction_upsampled)

LgR_upsampled_valid = model_LgR_upsampled.predict_proba(features_valid)[:, 1]
auc_roc_LgR = roc_auc_score(target_valid, LgR_upsampled_valid)
auc_roc_LgR

Полнота 0.7046728971962617
Точность 0.3964248159831756
F1-мера 0.5074024226110363


0.7634596085705452

Вывод

Показаели всех моделей улучшились.

Лучшие результаты показывает алгоритм случайный лес (RandomForestClassifier). На валидационной выборке RandomForestClassifier уже показывает резульаты F1 меры = 0.63, что выше целевого целевого значения.

## Тестирование модели

In [27]:
model_RFC_final = RandomForestClassifier(
    bootstrap = True, class_weight = 'balanced', max_depth= 7,  n_estimators = 43, random_state=12345)
model_RFC_final.fit(features_train_upsampled, target_train_upsampled)
model_RFC_final_prediction = model_RFC_final.predict(features_test)

rec_prec_f1(target_test, model_RFC_final_prediction)

model_RFC_final_valid = model_RFC_final.predict_proba(features_test)[:, 1]
auc_roc_RFC = roc_auc_score(target_test, model_RFC_final_valid)
auc_roc_RFC

Полнота 0.7707412862052038
Точность 0.5553590378493102
F1-мера 0.6455592105263157


0.8890239226821695

Выводы
В первоначальные данных наблюдался значительный дисбаланс (80% ответов целевого признака были негативными и только 20% позитивными), из-за чего обученная на этих данных модель не проходила проверку на адекватность. Все модели не первоначальных данных характеризовались высокой степенью ошибок и низким качеством взвешенной величины (F1) — модели показывали низкие результаты точности и полноты.

Мы устранили дисбаланс классов в обучающей выборки методом upsampling — увеличили количество значений позитивного класса в 4 раза. Так мы достигли баланса классо в обучеющей выборки: 0 - 0.499584 1 - 0.500416

Разобрали несколько вариантов борьбы с дисбалансом upsampling и downsampling

На новых данных все модели показали результат выше, чем на несбалансированной выборке. Лучшие показатели были у модели случайного леса:

Полнота 0.7401869158878505
Точность 0.5462068965517242
F1-мера 0.6285714285714287
AUC-ROC 0.859615704739483
Финальная модель прошла проверку на адекватность и ее значения:

Полнота 0.7707412862052038
Точность 0.5553590378493102
F1-мера 0.6455592105263157
AUC-ROC 0.8890239226821695